In [1]:
# Install latest version sklearn (0.23)
!pip install -U scikit-learn

     |████████████████████████████████| 22.2MB 68.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
# Imports
import pandas as pd
import numpy as np
import os
import shutil
from collections import Counter
import matplotlib.pyplot as plt
import time
import gc
import sys
import random
import pickle

from scipy.stats import iqr
from sklearn import preprocessing

import datetime
import logging

# Deep learning
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

from IPython.display import display
# Set pandas options
pd.options.display.max_rows = 2000

In [3]:
# Global variables 
THR_E = 100 # interaction threshold of exercises (E) for user
BATCH_SIZE = 128
TRAIN_FRACTION = 0.95

AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 160
N_SELECT_PER_EPOCH = 100000 # Random select N samples for each epoch from train/val set
VAL_EVERY_N_EPOCHS = 5
PRINT_EVERY_N_BATCHES = 50 
ZERO_TASK_ETC = True

SAVE_DICTS = True

In [4]:
# SEED THE EXPERIMENTS
np.random.seed(18)

In [5]:
# Experiment date and number!
date = datetime.datetime.today().strftime("%d-%b")
experiment = "-riiid-3"
OUTPUT_FOLDER = date + experiment 
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [6]:
# Selected features
selections = ["E", "r", "etc", "ltg", "at", "p", "ex"]
assert((selections[0] == "E") & (selections[1]=="r"))

In [7]:
# Choose model setting
ENC_EMB, ENC_DENSE = [0, 4, 5], [] 
DEC_EMB, DEC_DENSE = [1, 2, 3, 6], []

In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDER_FEATHER = "/content/drive/My Drive/kaggle-riiid/feather-files"
PREPROCESS_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/demo-riiid-preprocessing.ipynb"
MODEL_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/demo-riiid-transformer.ipynb" # https://stackoverflow.com/questions/57464810/how-to-run-a-jupyter-notebook-with-space-in-relative-path-from-another-notebook
# MODEL_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/riiid-functional-transformer.ipynb" # https://stackoverflow.com/questions/57464810/how-to-run-a-jupyter-notebook-with-space-in-relative-path-from-another-notebook


Mounted at /content/drive


In [9]:
# Add functions to preprocess data
%run $PREPROCESS_FILE

In [10]:
# Create logger
logging = create_logging(OUTPUT_FOLDER)

In [11]:
%%time
# Read all dataframes from feather files and print out
train = read_df_print(os.path.join(FOLDER_FEATHER, "train.feather")) 
questions = read_df_print(os.path.join(FOLDER_FEATHER, "questions.feather"))
lectures = read_df_print(os.path.join(FOLDER_FEATHER, "lectures.feather"))

(101230332, 10)


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,None
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False


(13523, 5)


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92


(418, 4)


,lecture_id,tag,part,type_of
0,89,24584,5,concept
1,100,22243,1,concept
2,185,7035,6,concept


CPU times: user 1.69 s, sys: 2.07 s, total: 3.76 s
Wall time: 29.5 s


In [12]:
train["content_type_id"].sum()

1959032

In [13]:
# train["user_id"].value_counts().hist()

In [14]:
# Preprocess for lectures
if ("v" in selections) or ("l" in selections) or ("vc" in selections):
    lectures["lecture_id"] = pd.Categorical(lectures["lecture_id"])
    lectures["new_id"] = lectures["lecture_id"].cat.codes
    lec_map = dict(zip(lectures["lecture_id"], lectures["new_id"]))

    train["l"] = train["content_id"]
    boolean_lec = train["content_type_id"]==1
    train.loc[boolean_lec, "l"] = train.loc[boolean_lec, "l"].map(lec_map)
    train["v"] = train["content_type_id"].shift(1).fillna(0).astype(np.int8)
    train["l"] = train["l"].shift(1).fillna(418).astype(np.int32)
    train.loc[train["v"]==0, "l"] = 418 # Start token value
    train["vc"] = train.groupby("user_id")["v"].cumsum()

In [15]:
# We don't need the lectures now!
train = train.loc[train["content_type_id"]==0]

31-12 11:46 numexpr.utils INFO     NumExpr defaulting to 4 threads.


In [16]:
# Save space by deleting some columns
del train["user_answer"]

In [17]:
N_questions, N_parts = questions["question_id"].nunique(), questions["part"].nunique()
N_response, N_task, N_lag, N_et, N_groups, N_attempt, N_avg, N_ltg, N_l, N_pos = 2, 2, 1440, 301, 10, 8, 101, 4000, 418, 1000 # N_avg = 100 for regular?

In [18]:
# Seq len of users [sorted same as df]. USED FOR TRAINING
seq_len_dict = dict(train["user_id"].value_counts())
users = train["user_id"].unique()
seq_lens = [seq_len_dict[user] for user in users]

In [19]:
# Train/valid split
train_len = int(len(users)*TRAIN_FRACTION)
val_len = len(users) - train_len
train_users, val_users = users[:train_len], users[train_len:]

# PROBS of sequence
seq_len_train, seq_len_val = seq_lens[:train_len], seq_lens[train_len:]
PROBS_TRAIN = seq_len_train/np.sum(seq_len_train)
PROBS_VAL = seq_len_val/np.sum(seq_len_val)

In [20]:
len(train_users), len(val_users)

(373973, 19683)

In [21]:
train["prior_question_had_explanation"].isnull().sum() # Less than users - why?

392506

## Add all inputs/outputs

In [22]:
p_lists, lt_lists, et_lists, tag_lists, task_lists, quantile_transformer_et, lt_cat, quantile_transformer_lt, et_cat, ltc, etc, ex, etg, ra, ca, at, r_dup, ltg, ltg_bins, v_lists, l_lists, n  = [], [], [], [], [], [], [], [], [], [],[], [], [], [], [], [], [], [], [], [], [], []

In [23]:
if "l" in selections:
  %time l_lists = return_l()
  del train["l"], train["v"], train["vc"]
if "v" in selections:
    %time v_lists = return_v()
    del train["l"], train["v"], train["vc"]

In [24]:
if "at" in selections:
    %time at = return_attempt()

CPU times: user 1min 6s, sys: 3.52 s, total: 1min 10s
Wall time: 1min 9s


In [25]:
n_bins = []
if "n" in selections:
  %time n, N_pos, n_bins = return_n(N_pos)

In [26]:
if "ra" in selections:
    %time ra = return_r_avg(add_start_token=True, N_avg = N_avg)

In [27]:
# Add exercises
%time E_lists = return_E()

CPU times: user 27.6 s, sys: 1.41 s, total: 29 s
Wall time: 29 s


In [28]:
# Add results, with or without start token
%time r_lists = return_r(add_start_token=True)

CPU times: user 28.9 s, sys: 279 ms, total: 29.1 s
Wall time: 29.1 s


In [29]:
# Lag time grouped
if "ltg" in selections:
    %time ltg, ltg_bins, N_ltg = return_ltg(N_ltg)    
print(N_ltg)

CPU times: user 47 s, sys: 734 ms, total: 47.8 s
Wall time: 46.9 s
678


In [30]:
if "etc" in selections:
    %time etc = return_etc(zero_task=ZERO_TASK_ETC)

CPU times: user 44.6 s, sys: 543 ms, total: 45.1 s
Wall time: 44.8 s


In [31]:
if "r_dup" in selections:
   %time r_dup = r_lists.apply(lambda x: x[:-1])

In [32]:
if "ca" in selections:
    %time ca = return_ca()

In [33]:
if "ex" in selections:
    %time ex = return_ex()

CPU times: user 40.8 s, sys: 386 ms, total: 41.2 s
Wall time: 41.2 s


In [34]:
if "ltc" in selections:
    %time ltc = return_ltc()

In [35]:
bins = list(range(30)) + [32, 35, 40, 48, 60, 80, 120, 299, 300]
if "etg" in selections:
    %time etg = return_etg(bins)
    # %time etg = return_etg(N_groups)


In [36]:
if "lt" in selections:
    %time lt_lists, quantile_transformer_lt = return_lt()

In [37]:
if "p" in selections:
    %time p_lists = return_p()

CPU times: user 23.8 s, sys: 189 ms, total: 24 s
Wall time: 24 s


In [38]:
if "et" in selections:
    %time et_lists, quantile_transformer_et = return_et()

In [39]:
if "tag" in selections:
    %time tag_lists = return_N_highest_tags() # TODO: specify N tags

In [40]:
if "task" in selections:
    %time task_lists = return_task_binary()

In [41]:
feature_mapping = {"E": E_lists, "r": r_lists, "p": p_lists, "et": et_lists, "lt": lt_lists, "tag": tag_lists, "task": task_lists, "et_std": et_lists, "ltc": ltc, "etc": etc, "ex": ex, "etg": etg, "ra": ra, "at": at, "ca": ca, "r_dup": r_dup, "v": v_lists, "ltg": ltg, "l": l_lists, "n": n}
type_mapping = {"E": tf.int32, "r": tf.int32, "p": tf.int32, "et": tf.float32, "et_std": tf.float32, "lt": tf.float32, "tag": tf.float32, "task": tf.float32, "ltc":tf.int32, "etc": tf.int32, "ex": tf.int32, "etg": tf.int32, "ra": tf.int32, "at": tf.int32, "ca": tf.int32, "r_dup": tf.int32, "v": tf.int32, "ltg": tf.int32, "l": tf.int32, "n": tf.int32}
pad_mapping = {"E": N_questions, "r": N_response, "p": 0, "et": 0.5, "et_std": 0.0, "lt": 0.5, "tag": 2.0, "task": float(N_task), "ltc": N_lag+1, "etc": N_et+1, "ex": 2, "etg": len(bins)+1, "ra": N_avg+1, "at": N_attempt, "ca": N_avg+1, "r_dup": N_response, "v": N_response, "ltg": N_ltg+2, "l": N_l+1, "n": N_pos+1}
vocab_mapping = {"E": N_questions+1, "r": N_response+2, "et":None, "p": N_parts+1, "lt": None, "tag": 3.0, "task": float(N_task+1), "et_std": float(300), "ltc": N_lag+2, "etc": N_et+2, "ex": 4, "etg": len(bins)+2, "ra": N_avg+2, "at": N_attempt+1, "ca": N_avg+2, "r_dup": N_response+2, "v": N_response+1, "ltg": N_ltg+3, "l": N_l+2, "n": N_pos+2}
pad_shapes = {"E": [THR_E], "r": [THR_E+1], "p": [THR_E], "et": [THR_E], "et_std": [THR_E],  "lt": [THR_E], "tag": [THR_E], "task": [THR_E], "ltc": [THR_E], "etc": [THR_E], "ex": [THR_E], "etg": [THR_E], "ra": [THR_E], "at": [THR_E], "ca": [THR_E], "r_dup": [THR_E], "v": [THR_E], "ltg": [THR_E], "l": [THR_E], "n": [THR_E]}

In [42]:
def save_as_dict(df, filename):
    seq_dict = df.to_dict()
    with open(os.path.join(OUTPUT_FOLDER_DICT, filename), 'wb') as handle:
      pickle.dump(seq_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
if SAVE_DICTS:
  OUTPUT_FOLDER_DICT = f"seq-dict-{THR_E}"
  os.makedirs(OUTPUT_FOLDER_DICT, exist_ok=True)

  for feature in selections:
      feat_df = feature_mapping[feature]
      feat_df = feat_df.apply(lambda x: x[-THR_E:]) # Take last THR_E in history 
      filename = f"{feature}.pickle"
      save_as_dict(feat_df, filename)
  shutil.move(OUTPUT_FOLDER_DICT, OUTPUT_FOLDER)

In [44]:
# # Add start token!
# # Answers
# r_lists = r_lists.apply(lambda x: [3] + x) # Start token = 3
# feature_mapping["r"] = r_lists

In [45]:
# Running answers
if "ra" in selections:
    ra = ra.apply(lambda x: x[:-1]) # For training => we don't have last value of average
    feature_mapping["ra"] = ra

In [46]:
vocab_sizes = [vocab_mapping[select] for select in selections]
feature_lists = [feature_mapping[select] for select in selections]

In [47]:
PADDING_VALUES = tuple((pad_mapping[select] for select in selections))
OUTPUT_TYPES = tuple((type_mapping[select] for select in selections))
PADDED_SHAPES = tuple((pad_shapes[select] for select in selections))

In [48]:
feature_lists

[user_id
 115           [5692, 5716, 128, 7860, 7922, 156, 51, 50, 789...
 124           [7900, 7876, 175, 1278, 2064, 2063, 2065, 3364...
 2746          [5273, 758, 5976, 236, 404, 382, 405, 873, 531...
 5382          [5000, 3944, 217, 5844, 5965, 4990, 5235, 6050...
 8623          [3915, 4750, 6456, 3968, 6104, 5738, 6435, 549...
                                     ...                        
 2147470770    [7900, 7876, 175, 1278, 2064, 2065, 2063, 3363...
 2147470777    [7900, 7876, 175, 1278, 2065, 2064, 2063, 3365...
 2147481750    [4137, 1270, 9261, 8201, 367, 378, 214, 6071, ...
 2147482216    [3748, 4765, 5474, 9261, 4665, 5987, 6666, 561...
 2147482888    [6147, 4792, 5738, 6102, 4748, 7956, 6435, 928...
 Name: content_id, Length: 393656, dtype: object, user_id
 115           [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, ...
 124           [3, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, ...
 2746          [3, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, ...
 5382          [3, 1, 0

In [49]:
try:
  del train
except Exception as e:
  print("train already deleted")
gc.collect(), gc.collect()

(0, 0)

# Create train/val datasets

In [50]:
# Lists of numpy arrays
train_list = [features.values[:train_len] for features in feature_lists]
val_list = [features.values[train_len:] for features in feature_lists]

In [51]:
def select_window_size(*x):
    series_len = len(x[0]) # Length of series
    if series_len <= THR_E: # Just return the sequence!
        return x
    else: # Random select from sequence
        max_select = series_len - THR_E
        random_select = tf.random.uniform(shape=(), minval=0, maxval=max_select, dtype=tf.int32)
        x = [i[random_select:random_select+THR_E] for i in x]
        x = tuple(x)
        return x

In [52]:
train_list

[array([list([5692, 5716, 128, 7860, 7922, 156, 51, 50, 7896, 7863, 152, 104, 108, 7900, 7901, 7971, 25, 183, 7926, 7927, 4, 7984, 45, 185, 55, 7876, 6, 172, 7898, 175, 100, 7859, 57, 7948, 151, 167, 7897, 7882, 7962, 1278, 2065, 2064, 2063, 3363, 3365, 3364]),
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3365, 3363, 2948, 2947, 2946, 2595, 2593, 2594, 4492, 4120, 4696, 6116, 6173, 6370, 6909, 6910, 6908, 6911, 7218, 7216, 7217, 7219]),
        list([5273, 758, 5976, 236, 404, 382, 405, 873, 531, 775, 294, 714, 297, 297, 775, 1295, 10684, 1014, 484]),
        ...,
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3365, 3363, 2947, 2948, 2946, 2593, 2595, 2594, 4492, 4120, 4696, 6116, 6173, 6370, 6878, 6880, 6879, 6877, 7219, 7218, 7217, 7216, 6106, 4755, 9313, 3586, 4476, 6432, 5845, 9094, 6191, 5437]),
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3363, 3365, 2946, 2948, 2947, 2594, 2595, 2593, 4492, 4120, 4696, 6116, 6173, 6370, 6879, 6880, 6

In [53]:
def create_train_dataset(N_training_per_epoch, probs_train):
  index = np.random.choice(train_len, N_training_per_epoch, replace=True, p = probs_train) # random indexing (similar to shuffling)
  tr_list = [features[index] for features in train_list] # Selecting by index 
  train_ds = (tf.data.Dataset
                  .from_generator(lambda: iter(zip(*tr_list)), output_types=OUTPUT_TYPES)
                  .map(select_window_size)
                  .padded_batch(batch_size = BATCH_SIZE, padded_shapes = PADDED_SHAPES, padding_values = PADDING_VALUES)
                  .prefetch(AUTO)
  )
  return train_ds

In [54]:
def create_val_dataset(N_valid_per_epoch, probs_val):
  index = np.random.choice(val_len, N_valid_per_epoch, replace=True, p = probs_val) # random indexing (similar to shuffling)
  vl_list = [features[index] for features in val_list] # Selecting by index 
  val_ds = (tf.data.Dataset
                  .from_generator(lambda: iter(zip(*vl_list)), output_types=OUTPUT_TYPES)
                  .map(select_window_size)
                  .padded_batch(batch_size = BATCH_SIZE, padded_shapes = PADDED_SHAPES, padding_values = PADDING_VALUES)
                  .cache()
                  .prefetch(AUTO)
  )
  return val_ds

In [55]:
val_dataset = create_val_dataset(N_SELECT_PER_EPOCH, PROBS_VAL)
a = iter(val_dataset) # Small check

In [56]:
next(a)

(<tf.Tensor: shape=(128, 100), dtype=int32, numpy=
 array([[ 1252,   322,  1725, ...,  3507,  3509,  3508],
        [10001,  5616,  6154, ...,  4105,  5563,  6076],
        [ 6630,  6720,  6717, ...,  2240,  2241,  2218],
        ...,
        [  930,   533,  1146, ...,   647,   585,   835],
        [ 9165,  4978,  4240, ...,  9365,  6016,  4481],
        [ 8249,   249,  6099, ..., 13523, 13523, 13523]], dtype=int32)>,
 <tf.Tensor: shape=(128, 101), dtype=int32, numpy=
 array([[1, 1, 1, ..., 1, 1, 2],
        [1, 1, 1, ..., 1, 1, 2],
        [1, 0, 1, ..., 0, 0, 2],
        ...,
        [1, 1, 1, ..., 0, 1, 2],
        [1, 1, 1, ..., 1, 1, 2],
        [3, 0, 1, ..., 2, 2, 2]], dtype=int32)>,
 <tf.Tensor: shape=(128, 100), dtype=int32, numpy=
 array([[ 17,  15,  19, ...,  30,   0,   0],
        [  7,  30,  25, ...,   8,  22, 294],
        [  0,  31,   0, ...,   0,   0,   4],
        ...,
        [ 21,  25,  17, ...,  27,  23,  18],
        [ 25,   8,  19, ...,  12,  13,  20],
        [30

## Get model and set hyperparameters

In [57]:
%run $MODEL_FILE

In [58]:
shutil.copy('/content/drive/My Drive/Colab Notebooks/demo-riiid-transformer.ipynb', OUTPUT_FOLDER)

'31-Dec-riiid-3/demo-riiid-transformer.ipynb'

In [59]:
# Hyperparameters main model
config = {"features": selections, "n_features": len(selections), "vocab_sizes": vocab_sizes, 
          "enc_emb": ENC_EMB, "enc_dense": ENC_DENSE, "dec_emb": DEC_EMB, "dec_dense": DEC_DENSE, 
          "window_size": THR_E, "enc_num_layers": 2, "dec_num_layers": 2, "d_model": 256, 
          "dff": 512, "num_heads": 8, "dropout_rate": 0.1, "pos_encoding": True,
          "padding_values": PADDING_VALUES, "output_types": OUTPUT_TYPES, "padded_shapes": PADDED_SHAPES, 
          "quantile_transformer_et": quantile_transformer_et,  "quantile_transformer_lt": quantile_transformer_lt, "ltg_bins": ltg_bins, "zero_task_etc": ZERO_TASK_ETC,
          "n_bins": n_bins}

with open(os.path.join(OUTPUT_FOLDER, "config.pickle"), 'wb') as handle:
  pickle.dump(config, handle, protocol=pickle.HIGHEST_PROTOCOL)

logging.info('config: %s', config)

31-12 11:52 root         INFO     config: {'features': ['E', 'r', 'etc', 'ltg', 'at', 'p', 'ex'], 'n_features': 7, 'vocab_sizes': [13524, 4, 303, 681, 9, 8, 4], 'enc_emb': [0, 4, 5], 'enc_dense': [], 'dec_emb': [1, 2, 3, 6], 'dec_dense': [], 'window_size': 100, 'enc_num_layers': 2, 'dec_num_layers': 2, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'pos_encoding': True, 'padding_values': (13523, 2, 302, 680, 8, 0, 2), 'output_types': (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32), 'padded_shapes': ([100], [101], [100], [100], [100], [100], [100]), 'quantile_transformer_et': [], 'quantile_transformer_lt': [], 'ltg_bins': array([0.00000000e+00, 2.00000000e+00, 3.00000000e+00, 4.00000000e+00,
       5.00000000e+00, 6.00000000e+00, 7.00000000e+00, 8.00000000e+00,
       9.00000000e+00, 1.00000000e+01, 1.10000000e+01, 1.20000000e+01,
       1.30000000e+01, 1.40000000e+01, 1.50000000e+01, 1.60000000e+01,
       1.70000000e+01, 1.80000000e+01, 1.90000

In [60]:
transformer = create_model_separate_input(config)
# tf.keras.utils.plot_model(transformer, os.path.join(OUTPUT_FOLDER, "model_plot.png"), show_shapes=True)

## Optimizer

In [61]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [62]:
learning_rate = CustomSchedule(config["d_model"])
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.999, 
                                      epsilon=1e-9)

## Loss and metrics

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [63]:
# Loss and metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

accuracy_object = tf.keras.metrics.sparse_categorical_accuracy

train_auc= tf.keras.metrics.AUC()
val_auc= tf.keras.metrics.AUC()

In [64]:
def loss_function(real, pred): # batch_size x seq_size x 1 vs  batch_size x seq_size x 3
    mask = tf.math.logical_not(tf.math.equal(tf.squeeze(real), N_response)) # batch_size x seq_size
    loss_ = loss_object(real, pred) # batch_size x seq_size
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    loss_ *= mask 
    loss_ = tf.reduce_sum(loss_)/tf.reduce_sum(mask) #loss becomes one value! (from all batches)
    return loss_

In [65]:
def metric_function(real, pred, auc_object): # batch_size x seq_size x 1 vs  batch_size x seq_size x 3
    # Predict accuracy
    mask = tf.math.logical_not(tf.math.equal(tf.squeeze(real), N_response)) # batch_size x seq_size
    accuracy = accuracy_object(real, pred) # batch_size x seq_size
    mask = tf.cast(mask, dtype=accuracy.dtype)
    
    accuracy *= mask
    accuracy = 100*tf.reduce_sum(accuracy)/tf.reduce_sum(mask)
    
    # A work-around to predict AUC => is it stable?
    pred = tf.nn.softmax(pred)
    pred = pred[:,:,1] # pred that answer is correct
    real = tf.keras.backend.flatten(real)
    pred = tf.keras.backend.flatten(pred)
    
    idxs = tf.math.logical_not(tf.math.equal(real, N_response))
    real = real[idxs]
    pred = pred[idxs]
    auc = auc_object(real, pred)
    return accuracy

In [66]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.Mean(name='val_accuracy')

## Training and checkpointing

Create the checkpoint path and the checkpoint manager. This will be used to save checkpoints every `n` epochs.

In [67]:
checkpoint_path = os.path.join(OUTPUT_FOLDER, "checkpoints/")
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

# https://stackoverflow.com/questions/62919208/how-to-restore-a-specific-checkpoint-in-tensorflow2-to-implement-early-stopping

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [68]:
# train_step_signature = [
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
#     tf.TensorSpec(shape=(None, THR_E), dtype=tf.int32),
# ]

In [69]:
@tf.function()
def train_step(tar_real, *inputs):
    
    with tf.GradientTape() as tape: 
        predictions = transformer(inputs, training=True)
        tar_real = tf.expand_dims(tar_real, -1) # IMPORTANT! DOESN'T WORK WITHOUT IT. ALWAYS GIVES ERROR INCOMPATIBLE SHAPE. E.G. (32,169) vs. (32,169,3)
        
        loss = loss_function(tar_real, predictions)
        accuracy = metric_function(tar_real, predictions, auc_object = train_auc)
    
    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy)

@tf.function()
def val_step(tar_real, *inputs):
    predictions = transformer(inputs, training=False)
    tar_real = tf.expand_dims(tar_real, -1) # IMPORTANT! DOESN'T WORK WITHOUT IT. ALWAYS GIVES ERROR INCOMPATIBLE SHAPE. E.G. (32,169) vs. (32,169,3)
    
    loss = loss_function(tar_real, predictions)
    accuracy = metric_function(tar_real, predictions, auc_object = val_auc)

    val_loss(loss)
    val_accuracy(accuracy)

In [ ]:
best_auc = 0
for epoch in range(EPOCHS):
    train_dataset = create_train_dataset(N_SELECT_PER_EPOCH, PROBS_TRAIN) # create train for each epoch
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    train_auc.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    val_auc.reset_states()

    # Train
    for (batch, (features)) in enumerate(train_dataset):
        E, tar, rest = features[0], features[1], features[2:] 
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        inputs = [E, tar_inp] + list(rest)        
        train_step(tar_real, inputs)
        
        if batch % PRINT_EVERY_N_BATCHES == 0:
            print ('Epoch {} TRAIN Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result(), train_auc.result()))

    logging.info('Epoch TRAIN {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result(),
                                                train_auc.result()
                                                ))
    
    # Validate and save model
    if (epoch + 1) % VAL_EVERY_N_EPOCHS == 0: 
      for (batch, (features)) in enumerate(val_dataset):
          E, tar, rest = features[0], features[1], features[2:] 
          tar_inp = tar[:, :-1]
          tar_real = tar[:, 1:]

          inputs = [E, tar_inp] + list(rest)          
          val_step(tar_real, inputs)

          if batch % PRINT_EVERY_N_BATCHES == 0:
              print ('Epoch {} VAL Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
                epoch + 1, batch, val_loss.result(), val_accuracy.result(), val_auc.result()))
      
      logging.info('Epoch VAL {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(epoch + 1, 
                                          val_loss.result(), 
                                          val_accuracy.result(),
                                          val_auc.result()
                                          ))    
       
      if val_auc.result() > best_auc:
          best_auc = val_auc.result()
          ckpt_save_path = ckpt_manager.save()
          logging.info('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                            ckpt_save_path))
        
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 TRAIN Batch 0 Loss 1.5221 Accuracy 17.0190 AUC 0.4985
Epoch 1 TRAIN Batch 50 Loss 0.9485 Accuracy 50.5526 AUC 0.5038
Epoch 1 TRAIN Batch 100 Loss 0.8259 Accuracy 56.0689 AUC 0.5037
Epoch 1 TRAIN Batch 150 Loss 0.7767 Accuracy 58.4479 AUC 0.5049
Epoch 1 TRAIN Batch 200 Loss 0.7484 Accuracy 59.9272 AUC 0.5063
Epoch 1 TRAIN Batch 250 Loss 0.7295 Accuracy 60.9893 AUC 0.5078
Epoch 1 TRAIN Batch 300 Loss 0.7156 Accuracy 61.8328 AUC 0.5096
Epoch 1 TRAIN Batch 350 Loss 0.7048 Accuracy 62.5083 AUC 0.5119
Epoch 1 TRAIN Batch 400 Loss 0.6963 Accuracy 63.0206 AUC 0.5154
Epoch 1 TRAIN Batch 450 Loss 0.6891 Accuracy 63.4237 AUC 0.5201
Epoch 1 TRAIN Batch 500 Loss 0.6829 Accuracy 63.7858 AUC 0.5257
Epoch 1 TRAIN Batch 550 Loss 0.6776 Accuracy 64.0833 AUC 0.5311
Epoch 1 TRAIN Batch 600 Loss 0.6729 Accuracy 64.3537 AUC 0.5365
Epoch 1 TRAIN Batch 650 Loss 0.6688 Accuracy 64.5765 AUC 0.5417
Epoch 1 TRAIN Batch 700 Loss 0.6652 Accuracy 64.7773 AUC 0.5465
Epoch 1 TRAIN Batch 750 Loss 0.6618 Accurac

31-12 11:55 root         INFO     Epoch TRAIN 1 Loss 0.6601 Accuracy 65.0604 AUC 0.5535


Time taken for 1 epoch: 210.32253122329712 secs

Epoch 2 TRAIN Batch 0 Loss 0.6137 Accuracy 67.5559 AUC 0.6228
Epoch 2 TRAIN Batch 50 Loss 0.6145 Accuracy 67.4872 AUC 0.6209
Epoch 2 TRAIN Batch 100 Loss 0.6129 Accuracy 67.5833 AUC 0.6251
Epoch 2 TRAIN Batch 150 Loss 0.6119 Accuracy 67.6615 AUC 0.6269
Epoch 2 TRAIN Batch 200 Loss 0.6113 Accuracy 67.6970 AUC 0.6276
Epoch 2 TRAIN Batch 250 Loss 0.6105 Accuracy 67.7266 AUC 0.6297
Epoch 2 TRAIN Batch 300 Loss 0.6100 Accuracy 67.7169 AUC 0.6315
Epoch 2 TRAIN Batch 350 Loss 0.6093 Accuracy 67.7393 AUC 0.6327
Epoch 2 TRAIN Batch 400 Loss 0.6089 Accuracy 67.7464 AUC 0.6336
Epoch 2 TRAIN Batch 450 Loss 0.6085 Accuracy 67.7594 AUC 0.6343
Epoch 2 TRAIN Batch 500 Loss 0.6082 Accuracy 67.7626 AUC 0.6352
Epoch 2 TRAIN Batch 550 Loss 0.6078 Accuracy 67.7641 AUC 0.6362
Epoch 2 TRAIN Batch 600 Loss 0.6073 Accuracy 67.7865 AUC 0.6373
Epoch 2 TRAIN Batch 650 Loss 0.6068 Accuracy 67.8159 AUC 0.6385
Epoch 2 TRAIN Batch 700 Loss 0.6063 Accuracy 67.8311 AUC 0

31-12 11:59 root         INFO     Epoch TRAIN 2 Loss 0.6055 Accuracy 67.8641 AUC 0.6420


Time taken for 1 epoch: 202.99348497390747 secs

Epoch 3 TRAIN Batch 0 Loss 0.5956 Accuracy 67.7794 AUC 0.6865
Epoch 3 TRAIN Batch 50 Loss 0.5741 Accuracy 70.1881 AUC 0.7074
Epoch 3 TRAIN Batch 100 Loss 0.5652 Accuracy 70.8922 AUC 0.7200
Epoch 3 TRAIN Batch 150 Loss 0.5587 Accuracy 71.3665 AUC 0.7295
Epoch 3 TRAIN Batch 200 Loss 0.5544 Accuracy 71.6473 AUC 0.7356
Epoch 3 TRAIN Batch 250 Loss 0.5517 Accuracy 71.8241 AUC 0.7398
Epoch 3 TRAIN Batch 300 Loss 0.5494 Accuracy 71.9612 AUC 0.7428
Epoch 3 TRAIN Batch 350 Loss 0.5477 Accuracy 72.0615 AUC 0.7453
Epoch 3 TRAIN Batch 400 Loss 0.5460 Accuracy 72.1791 AUC 0.7473
Epoch 3 TRAIN Batch 450 Loss 0.5444 Accuracy 72.2877 AUC 0.7491
Epoch 3 TRAIN Batch 500 Loss 0.5432 Accuracy 72.3635 AUC 0.7507
Epoch 3 TRAIN Batch 550 Loss 0.5426 Accuracy 72.3915 AUC 0.7516
Epoch 3 TRAIN Batch 600 Loss 0.5418 Accuracy 72.4433 AUC 0.7526
Epoch 3 TRAIN Batch 650 Loss 0.5413 Accuracy 72.4761 AUC 0.7533
Epoch 3 TRAIN Batch 700 Loss 0.5405 Accuracy 72.5168 AUC 0

...

Time taken for 1 epoch: 197.3580710887909 secs

Epoch 67 TRAIN Batch 0 Loss 0.5062 Accuracy 74.4856 AUC 0.7916
Epoch 67 TRAIN Batch 50 Loss 0.4991 Accuracy 74.9627 AUC 0.7992
Epoch 67 TRAIN Batch 100 Loss 0.4994 Accuracy 74.9492 AUC 0.7993
Epoch 67 TRAIN Batch 150 Loss 0.4989 Accuracy 74.9532 AUC 0.7998
Epoch 67 TRAIN Batch 200 Loss 0.4986 Accuracy 74.9770 AUC 0.8000
Epoch 67 TRAIN Batch 250 Loss 0.4987 Accuracy 74.9520 AUC 0.7999
Epoch 67 TRAIN Batch 300 Loss 0.4988 Accuracy 74.9396 AUC 0.7998
Epoch 67 TRAIN Batch 350 Loss 0.4989 Accuracy 74.9346 AUC 0.7997
Epoch 67 TRAIN Batch 400 Loss 0.4987 Accuracy 74.9431 AUC 0.7998
Epoch 67 TRAIN Batch 450 Loss 0.4987 Accuracy 74.9464 AUC 0.7997
Epoch 67 TRAIN Batch 500 Loss 0.4988 Accuracy 74.9377 AUC 0.7997
Epoch 67 TRAIN Batch 550 Loss 0.4986 Accuracy 74.9530 AUC 0.7998
Epoch 67 TRAIN Batch 600 Loss 0.4985 Accuracy 74.9556 AUC 0.7999
Epoch 67 TRAIN Batch 650 Loss 0.4986 Accuracy 74.9524 AUC 0.7999
Epoch 67 TRAIN Batch 700 Loss 0.4985 Accuracy

31-12 15:40 root         INFO     Epoch TRAIN 67 Loss 0.4986 Accuracy 74.9458 AUC 0.7999


Time taken for 1 epoch: 197.3019745349884 secs

Epoch 68 TRAIN Batch 0 Loss 0.5033 Accuracy 74.6315 AUC 0.7970
Epoch 68 TRAIN Batch 50 Loss 0.4964 Accuracy 75.1355 AUC 0.8020
Epoch 68 TRAIN Batch 100 Loss 0.4965 Accuracy 75.1286 AUC 0.8017
Epoch 68 TRAIN Batch 150 Loss 0.4968 Accuracy 75.0942 AUC 0.8010
Epoch 68 TRAIN Batch 200 Loss 0.4969 Accuracy 75.0728 AUC 0.8009
Epoch 68 TRAIN Batch 250 Loss 0.4973 Accuracy 75.0497 AUC 0.8009
Epoch 68 TRAIN Batch 300 Loss 0.4974 Accuracy 75.0441 AUC 0.8008
Epoch 68 TRAIN Batch 350 Loss 0.4974 Accuracy 75.0422 AUC 0.8007
Epoch 68 TRAIN Batch 400 Loss 0.4976 Accuracy 75.0290 AUC 0.8005
Epoch 68 TRAIN Batch 450 Loss 0.4978 Accuracy 75.0226 AUC 0.8004
Epoch 68 TRAIN Batch 500 Loss 0.4979 Accuracy 75.0082 AUC 0.8003
Epoch 68 TRAIN Batch 550 Loss 0.4981 Accuracy 74.9967 AUC 0.8003
Epoch 68 TRAIN Batch 600 Loss 0.4982 Accuracy 74.9868 AUC 0.8001
Epoch 68 TRAIN Batch 650 Loss 0.4982 Accuracy 74.9771 AUC 0.8001
Epoch 68 TRAIN Batch 700 Loss 0.4982 Accuracy

31-12 15:43 root         INFO     Epoch TRAIN 68 Loss 0.4981 Accuracy 74.9823 AUC 0.8001


Time taken for 1 epoch: 197.47794342041016 secs

Epoch 69 TRAIN Batch 0 Loss 0.5087 Accuracy 74.2871 AUC 0.7899
Epoch 69 TRAIN Batch 50 Loss 0.4987 Accuracy 74.9981 AUC 0.7993
Epoch 69 TRAIN Batch 100 Loss 0.4996 Accuracy 74.8977 AUC 0.7985


In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks(OUTPUT_FOLDER) #lists all downloadable files on server

## Add output to Drive

In [ ]:
OUTPUT_DRIVE = "/content/drive/My Drive/kaggle-riiid/subs"

In [ ]:
# https://stackoverflow.com/questions/15034151/copy-directory-contents-into-a-directory-with-python
from distutils.dir_util import copy_tree
copy_tree(OUTPUT_FOLDER, os.path.join(OUTPUT_DRIVE, OUTPUT_FOLDER))

## Add output to Kaggle

In [ ]:
KAGGLE_JSON = "/content/drive/My\ Drive/kaggle-riiid/kaggle.json"
!mkdir -p ~/.kaggle
!cp $KAGGLE_JSON ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets init -p {OUTPUT_FOLDER}

In [ ]:
import json

with open(f'{OUTPUT_FOLDER}/dataset-metadata.json', 'r+') as f:
    data = json.load(f)
    data['title'] = OUTPUT_FOLDER
    data['id'] = f'rafiko1/{OUTPUT_FOLDER}'
    f.seek(0)
    json.dump(data, f, indent=4)
    f.truncate()

!cat {OUTPUT_FOLDER}/dataset-metadata.json

In [ ]:
!kaggle datasets create -p {OUTPUT_FOLDER} -q -r zip 

In [ ]:
os.listdir(OUTPUT_FOLDER)

In [ ]:
ltg_bins

## Summary

In this tutorial, you learned about positional encoding, multi-head attention, the importance of masking and how to create a transformer.

Try using a different dataset to train the transformer. You can also create the base transformer or transformer XL by changing the hyperparameters above. You can also use the layers defined here to create [BERT](https://arxiv.org/abs/1810.04805) and train state of the art models. Futhermore, you can implement beam search to get better predictions.

# Older code

In [ ]:
# # From https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# class DataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, df, shuffle=True):
#         'Initialization'
        
#         self.df = df
#         self.users = self.df["user_id"].nunique()
#         self.batch_size = BATCH_SIZE
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.list_IDs) / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Find list of IDs
#         list_IDs_temp = [self.list_IDs[k] for k in indexes]

#         # Generate data
#         X, y = self.__data_generation(list_IDs_temp)

#         return X, y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.list_IDs))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, list_IDs_temp):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size), dtype=int)

#         # Generate data
#         for i, ID in enumerate(list_IDs_temp):
#             # Store sample
#             X[i,] = np.load('data/' + ID + '.npy')

#             # Store class
#             y[i] = self.labels[ID]

#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# def save_dict(seq_dict, filename):
#     with open(os.path.join(OUTPUT_FOLDER, filename), 'wb') as handle:
#       pickle.dump(seq_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Save last N dict to memory
# E_dict = E_lists.apply(lambda x: x[-200:]).to_dict()
# r_dict = r_lists.apply(lambda x: x[-200:]).to_dict()
# et_dict = et_lists.apply(lambda x: x[-200:]).to_dict()

In [ ]:
# save_dict(E_dict, "E.pkl")
# save_dict(r_dict, "r.pkl")
# save_dict(et_dict, "et.pkl")